In [ ]:
import os
import boto3
import awswrangler as wr
import pyodbc

def lambda_handler(event, context):
    # Configuración de conexión
    conn_str = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={os.environ['DB_HOST']};"
        f"DATABASE={os.environ['DB_NAME']};"
        f"UID={os.environ['DB_USER']};"
        f"PWD={os.environ['DB_PASS']}"
    )
    
    s3_path_clear = os.environ['S3_CLEAR_PATH']

    try:
        cnxn = pyodbc.connect(conn_str)
        cursor = cnxn.cursor()
        print("✅ Conexión establecida. Iniciando carga de 6 etapas...")

        # --- CARGA 1: PROVEEDOR (Dimension) ---
        df_vendors = wr.s3.read_parquet(path=f"{s3_path_clear}Facturas_compras/")
        wr.sqlserver.to_sql(df=df_vendors, con=cnxn, table="TempVendors", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.Proveedor (VendorNumber, VendorName)
            SELECT CAST(VendorNumber AS INT), MAX(VendorName)
            FROM dbo.TempVendors
            WHERE VendorNumber IS NOT NULL AND CAST(VendorNumber AS INT) NOT IN (SELECT VendorNumber FROM dbo.Proveedor)
            GROUP BY CAST(VendorNumber AS INT);
        """)

        # CARGA 2: STORES (Dimension) ---
        df_stores = wr.s3.read_parquet(path=f"{s3_path_clear}Inicio_inventario/")
        wr.sqlserver.to_sql(df=df_stores, con=cnxn, table="TempStores", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.Stores (StoreID, StoreName, City)
            SELECT DISTINCT CAST(Store AS INT), City, City
            FROM dbo.TempStores
            WHERE Store IS NOT NULL AND CAST(Store AS INT) NOT IN (SELECT StoreID FROM dbo.Stores);
        """)

        # CARGA 3: BRANDS (Dimension) ---
        df_brands = wr.s3.read_parquet(path=f"{s3_path_clear}Precios_compras_2017/")
        wr.sqlserver.to_sql(df=df_brands, con=cnxn, table="TempBrands", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.Brands (BrandID, BrandDescription, Size, Volume, Classification, RetailPrice, PurchasePrice, VendorNumber)
            SELECT TRY_CAST(Brand AS INT), COALESCE(NULLIF(Description, ''), 'Unknown'), Size, 
                   TRY_CAST(Volume AS INT), TRY_CAST(Classification AS INT), 
                   TRY_CAST(Price AS DECIMAL(10,2)), TRY_CAST(PurchasePrice AS DECIMAL(10,2)), TRY_CAST(VendorNumber AS INT)
            FROM dbo.TempBrands
            WHERE TRY_CAST(Brand AS INT) IS NOT NULL AND TRY_CAST(Brand AS INT) NOT IN (SELECT BrandID FROM dbo.Brands);
        """)

        # CARGA 4: INVENTORY (Fact - Consolidada) ---
        df_beg = wr.s3.read_parquet(path=f"{s3_path_clear}Inicio_inventario/")
        df_end = wr.s3.read_parquet(path=f"{s3_path_clear}Final_inventario/")
        wr.sqlserver.to_sql(df=df_beg, con=cnxn, table="TempBegInv", schema="dbo", mode="overwrite")
        wr.sqlserver.to_sql(df=df_end, con=cnxn, table="TempEndInv", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.Inventory (InventoryID, StoreID, BrandID, OnHandBegin, OnHandEnd, StartDate, EndDate)
            SELECT COALESCE(b.InventoryId, e.InventoryId), CAST(COALESCE(b.Store, e.Store) AS INT), 
                   CAST(COALESCE(b.Brand, e.Brand) AS INT), TRY_CAST(b.onHand AS INT), TRY_CAST(e.onHand AS INT), 
                   b.startDate, e.endDate
            FROM dbo.TempBegInv b
            FULL OUTER JOIN dbo.TempEndInv e ON b.InventoryId = e.InventoryId
            WHERE COALESCE(b.InventoryId, e.InventoryId) IS NOT NULL 
            AND COALESCE(b.InventoryId, e.InventoryId) NOT IN (SELECT InventoryID FROM dbo.Inventory);
        """)

        # CARGA 5: SALESDETAIL (Fact) ---
        df_sales = wr.s3.read_parquet(path=f"{s3_path_clear}Venta_final/")
        wr.sqlserver.to_sql(df=df_sales, con=cnxn, table="TempSales", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.SalesDetail (InventoryID, StoreID, BrandID, VendorNumber, SalesQuantity, SalesDollars, SalesPrice, SalesDate, Volume, Classification, ExciseTax)
            SELECT InventoryId, CAST(Store AS INT), CAST(Brand AS INT), CAST(VendorNo AS INT), 
                   TRY_CAST(SalesQuantity AS INT), TRY_CAST(SalesDollars AS DECIMAL(10,2)), 
                   TRY_CAST(SalesPrice AS DECIMAL(10,2)), SalesDate, TRY_CAST(Volume AS INT), 
                   TRY_CAST(Classification AS INT), TRY_CAST(ExciseTax AS DECIMAL(10,2))
            FROM dbo.TempSales WHERE InventoryId IS NOT NULL;
        """)

        # CARGA 6: PURCHASES (Fact - ¡La que faltaba!) ---
        df_purch = wr.s3.read_parquet(path=f"{s3_path_clear}Compra_final/")
        wr.sqlserver.to_sql(df=df_purch, con=cnxn, table="TempPurchases", schema="dbo", mode="overwrite")
        cursor.execute("""
            INSERT INTO dbo.Purchases (InventoryID, StoreID, BrandID, VendorNumber, PONumber, PODate, ReceivingDate, InvoiceDate, PayDate, PurchasePrice, Quantity, Dollars, Classification)
            SELECT InventoryId, TRY_CAST(Store AS INT), TRY_CAST(Brand AS INT), TRY_CAST(VendorNumber AS INT), 
                   TRY_CAST(PONumber AS INT), PODate, ReceivingDate, InvoiceDate, PayDate,
                   TRY_CAST(PurchasePrice AS DECIMAL(10,2)), TRY_CAST(Quantity AS INT), 
                   TRY_CAST(Dollars AS DECIMAL(10,2)), TRY_CAST(Classification AS INT)
            FROM dbo.TempPurchases WHERE InventoryId IS NOT NULL;
        """)

        cnxn.commit()
        print("Carga exitosa de las 6 tablas transaccionales y maestras.")
        return {"status": 200, "body": "Carga completa: 6/6 tablas procesadas."}

    except Exception as e:
        print(f"Error crítico: {str(e)}")
        raise e
    finally:
        if 'cnxn' in locals(): cnxn.close()